<a href="https://colab.research.google.com/github/HafsaI/CUDA_Spring2023/blob/main/Week2/OperationsOnNumbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wz7d6o6g
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wz7d6o6g
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=38bab01381fcb7fd9210801625ddf27e21a63c4bdfed9a7e7800e52653584b21
  Stored in directory: /tmp/pip-ephem-wheel-cache-jnjzda54/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
!nvidia-smi

Fri Jan 20 07:29:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
%%cu
#include <stdio.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

//kernel function definition
__global__ void add(int *a , int *b, int *c) 
{
  *c = *a + *b;
}
__global__ void sub(int *a , int *b, int *c) 
{
  *c =  *b - *a;
}

int main() {
  int h_a = 10;
  int h_b = 20;
  int h_c = 10 + 20;
  int h_d = 20 - 10;

  // need dynamic memory alocation on device
  int *d_a, *d_b, *d_c;

  //allocate memory on device, so need to store address of pointerin gpu, pointer ka address
  cudaMalloc((void **)&d_a, sizeof(int));
  cudaMalloc((void **)&d_b, sizeof(int));
  cudaMalloc((void **)&d_c, sizeof(int));

  //copy host data to device memory
  cudaMemcpy(d_a, &h_a, sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &h_b, sizeof(int), cudaMemcpyHostToDevice);
  
  //call kernel
  add<<<1,1>>>(d_a, d_b, d_c); 
  

  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU");
  printf("Addition Answer (on host): %d\n", h_c);
  printf("Subtraction Answer (on host): %d\n", h_d);

  //copy device data to host memory
  cudaMemcpy(&h_c, d_c, sizeof(int), cudaMemcpyDeviceToHost);
  printf("Addition Answer (on device): %d\n", h_c);

  sub<<<1,1>>>(d_a, d_b, d_c); 
  cudaMemcpy(&h_d, d_c, sizeof(int), cudaMemcpyDeviceToHost);
   printf("Subtraction Answer (on device): %d\n", h_d);

  //release GPU memory
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  
  return 0;
}

Addition Answer (on host): 30
Subtraction Answer (on host): 10
Addition Answer (on device): 30
Subtraction Answer (on device): 10

